In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import FastICA

from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVR
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from sklearn.model_selection import KFold, cross_val_score, train_test_split

from hyperopt import fmin, tpe, hp
from bayes_opt import BayesianOptimization

# SVR optimization methods

In [2]:
def bayesian_optimization_svr(X, y, cv=6, max_iter_svr=100, max_iter_opt=15):
    svr_opt = BayesianOptimization(
        lambda epsilon: cross_val_score(
            LinearSVR(C=100, epsilon=epsilon, max_iter=max_iter_svr),
            X, y.squeeze(), cv=KFold(n_splits=cv).split(X), scoring='neg_mean_squared_error'
        ).mean(),
        {'epsilon': (0.0, 5.0)},
        verbose=0
    )
    
    svr_opt.init(10)
    svr_opt.maximize(n_iter=max_iter_opt)
    
    return svr_opt.res['max']['max_params']['epsilon']

def hyperopt_optimization_svr(X, y, cv=6, max_iter_svr=100, max_iter_opt=15):
    space = hp.choice('regressor_type', [
        {
            'type': 'svr',
            'epsilon': hp.uniform('svr_epsilon', 0.0, 5.0)
        }
    ])

    best = fmin(
        fn=lambda args: cross_val_score(
            LinearSVR(C=100, epsilon=args['epsilon'], max_iter=max_iter_svr),
            X, y.squeeze(), cv=KFold(n_splits=cv).split(X), scoring='neg_mean_squared_error'
        ).mean(),
        space=space,
        algo=tpe.suggest,
        max_evals=max_iter_opt
    )

    return best['svr_epsilon']

def evaulate_best_svr_argument(X, y, eps_vals: list, cv=6):
    scores = []
    for e in eps_vals:
        scores.append(cross_val_score(
            LinearSVR(C=100, epsilon=e, max_iter=200),
            X, y.squeeze(), cv=KFold(n_splits=cv).split(X), scoring='neg_mean_squared_error'
        ).mean())
        
    return eps_vals[np.argmax(scores)]

# RF regressor optimization code

In [3]:
# hyperopt for lightgbm shows terrible results
def hyperopt_optimization_lightgbm(X, y, cv=6, max_iter_opt=15):
    space = hp.choice('regressor_type', [
        {
            'type': 'lightgbm',
            'feature_fraction': hp.uniform('feature_fraction', 0.05, 0.95),
            'bagging_fraction': hp.uniform('bagging_fraction', 0.05, 0.95),
            'bagging_freq': hp.uniform('bagging_freq', 1, 50),
            'n_estimators': hp.uniform('n_estimators', 5, 50),
            #'max_bin': hp.uniform('max_bin', )
        }
    ])

    best = fmin(
        fn=lambda args: cross_val_score(
            LGBMRegressor(
                boosting_type='rf', 
                feature_fraction=args['feature_fraction'], 
                bagging_freq=int(args['bagging_freq']), 
                bagging_fraction=args['bagging_fraction'],
                n_estimators=int(args['n_estimators'])
            ),
            X, y.squeeze(), cv=KFold(n_splits=cv).split(X), scoring='neg_mean_squared_error'
        ).mean(),
        space=space,
        algo=tpe.suggest,
        max_evals=max_iter_opt
    )
    
    return best

def bayesian_optimization_lightgbm(X, y, cv=6, max_iter_opt=15):
    svr_opt = BayesianOptimization(
        lambda feature_fraction, bagging_freq, bagging_fraction, n_estimators: cross_val_score(
            LGBMRegressor(
                boosting_type='rf', 
                feature_fraction=feature_fraction, 
                bagging_freq=int(bagging_freq), 
                bagging_fraction=bagging_fraction,
                n_estimators=int(n_estimators)
            ),
            X, y.squeeze(), cv=KFold(n_splits=cv).split(X), scoring='neg_mean_squared_error'
        ).mean(),
        {'feature_fraction': (0.05, 0.95),
         'bagging_fraction': (0.05, 0.95),
         'bagging_freq': (1, 50),
         'n_estimators': (5, 50) },
        verbose=0
    )
    
    svr_opt.init(10)
    svr_opt.maximize(n_iter=max_iter_opt)
    
    return svr_opt.res['max']['max_params']#['C']

## Facebook Comment Volume Dataset

In [86]:
df = pd.read_csv('../datasets/facebook_comments.csv', index_col=0)

In [87]:
print('dataset size: {}'.format(df.shape))

dataset size: (40949, 54)


In [88]:
# encoding categorical variables
# H Local - category
# Post Promotion Status - category
# Base Time - time variable
# Page Category - category

H_Local = OneHotEncoder().fit_transform(df['H Local'].values.reshape(-1, 1)).todense()
Post_Promotion_Status = OneHotEncoder().fit_transform(df['Post Promotion Status'].values.reshape(-1, 1)).todense()
Base_Time = OneHotEncoder().fit_transform(df['Base Time'].values.reshape(-1, 1)).todense()
Page_Category = OneHotEncoder().fit_transform(df['Page Category'].values.reshape(-1, 1)).todense()

In [89]:
y = df.Target.values.reshape(-1, 1)
X = df.drop(['H Local', 'Post Promotion Status', 'Base Time', 'Page Category', 'Target'], axis=1).values
X = np.hstack([X, H_Local, Post_Promotion_Status, Base_Time, Page_Category])

In [90]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

In [91]:
print('dataset size after preprocessing: {}'.format(X.shape))

dataset size after preprocessing: (40949, 228)


### Training model with ICA-processed data

In [92]:
# number of components is fixed to 10
ica = FastICA(n_components=10)
X_ica = ica.fit_transform(X_train)
X_val_ica = ica.transform(X_val)

In [93]:
X_ica, y_train = shuffle(X_ica, y_train)

### * Estimating SVR penalty

In [94]:
eps_hyperopt = hyperopt_optimization_svr(X_ica, y_train, cv=4, max_iter_svr=200, max_iter_opt=15)
eps_bayesian = bayesian_optimization_svr(X_ica, y_train, cv=4, max_iter_svr=200, max_iter_opt=15)

D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-10.96843993]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 48, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.17383541]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 52, 'nit': 4, '

In [95]:
eps_opt = evaulate_best_svr_argument(X_ica, y_train, [eps_hyperopt, eps_bayesian], cv=6)

### * Estimating LightGBM params

In [96]:
params_opt = bayesian_optimization_lightgbm(X_ica, y_train, cv=6, max_iter_opt=50)

D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.56812334]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 51, 'nit': 3, 'warnflag': 2}
  " state: %s" % convergence_dict)
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([10.30617905]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 51, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.16780138]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 62, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: User

In [97]:
params_opt['bagging_freq'] = int(params_opt['bagging_freq'])
params_opt['n_estimators'] = int(params_opt['n_estimators'])

### * Evaluate with CV

In [98]:
models = {
    'linear': LinearRegression(),
    'svr': LinearSVR(C=100, epsilon=eps_opt, max_iter=200),
    'forest': LGBMRegressor(boosting_type='rf', **params_opt)
}

In [99]:
stats = {}

for k, model in models.items():
    stats[k] = []
    kfold = KFold(n_splits=8, shuffle=True)
    
    for train_ix, test_ix in kfold.split(X_ica, y_train):
        X_crossval_train, X_crossval_test = X_ica[train_ix], X_ica[test_ix]
        y_crossval_train, y_crossval_test = y_train[train_ix], y_train[test_ix]
        
        # here must be sume sort of optimization
        model.fit(X_crossval_train, y_crossval_train.ravel())
        stats[k].append(mean_squared_error(model.predict(X_crossval_test), y_crossval_test))

In [100]:
print('for decorrelated data using ICA (averate scores)')
for model, model_stats in stats.items():
    print('{}, MSE: {}'.format(model, np.mean(model_stats)))

for decorrelated data using ICA (averate scores)
linear, MSE: 889.8961951122166
svr, MSE: 1051.161448878846
forest, MSE: 901.549447457496


In [101]:
print('for decorrelated data using ICA (validation scores):')
for name, model in models.items():
    model.fit(X_ica, y_train.squeeze())
    print('{}, MSE: {}'.format(name, mean_squared_error(model.predict(X_val_ica), y_val.squeeze())))

for decorrelated data using ICA (validation scores):
linear, MSE: 1196.9140843017153
svr, MSE: 1338.914875145344
forest, MSE: 1153.2025810390717


## Parkinsons Telemonitoring Data Set

In [102]:
df = pd.read_csv('../datasets/parkinsons_updrs.data')

In [103]:
print('dataset size: {}'.format(df.shape))

dataset size: (5875, 22)


In [104]:
subject = df['subject#'].unique().tolist()
subject_binary = np.zeros((df.shape[0], len(subject)))
for k, i in df.iterrows():
    subject_binary[k, int(i['subject#']) - 1] = 1

In [105]:
# replacing categorical features with binary values
y = df.total_UPDRS.values
X = df.drop(['motor_UPDRS', 'total_UPDRS', 'subject#'], axis=1).values
X = np.concatenate([subject_binary, X], axis=1)

In [106]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

In [107]:
print('dataset size after preprocessing: {}'.format(X.shape))

dataset size after preprocessing: (5875, 61)


### Training model with ICA-processed data

In [108]:
# number of components is fixed to 10
ica = FastICA(n_components=10)
X_ica = ica.fit_transform(X_train)
X_val_ica = ica.transform(X_val)

In [109]:
X_ica, y_train = shuffle(X_ica, y_train)

### * Estimating SVR penalty

In [110]:
eps_hyperopt = hyperopt_optimization_svr(X_ica, y_train, cv=4, max_iter_svr=200, max_iter_opt=15)
eps_bayesian = bayesian_optimization_svr(X_ica, y_train, cv=4, max_iter_svr=200, max_iter_opt=15)

D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([0.0316701]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 49, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([0.01177375]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 49, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-5.00753522e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 73, 'nit': 7, 'warnflag': 2}
  " state: %s" % convergence_dict)
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: Use

In [111]:
eps_opt = evaulate_best_svr_argument(X_ica, y_train, [eps_hyperopt, eps_bayesian], cv=6)

### * Estimating LightGBM params

In [112]:
params_opt = bayesian_optimization_lightgbm(X_ica, y_train, cv=6, max_iter_opt=50)

In [113]:
params_opt['bagging_freq'] = int(params_opt['bagging_freq'])
params_opt['n_estimators'] = int(params_opt['n_estimators'])

### * Evaluate with CV

In [114]:
models = {
    'linear': LinearRegression(),
    'svr': LinearSVR(C=100, epsilon=eps_opt, max_iter=200),
    'forest': LGBMRegressor(boosting_type='rf', **params_opt)
}

In [115]:
stats = {}

for k, model in models.items():
    stats[k] = []
    kfold = KFold(n_splits=8, shuffle=True)
    
    for train_ix, test_ix in kfold.split(X_ica, y_train):
        X_crossval_train, X_crossval_test = X_ica[train_ix], X_ica[test_ix]
        y_crossval_train, y_crossval_test = y_train[train_ix], y_train[test_ix]
        
        # here must be sume sort of optimization
        model.fit(X_crossval_train, y_crossval_train.ravel())
        stats[k].append(mean_squared_error(model.predict(X_crossval_test), y_crossval_test))

In [116]:
print('for decorrelated data using ICA')
for model, model_stats in stats.items():
    print('{}, MSE: {}'.format(model, np.mean(model_stats)))

for decorrelated data using ICA
linear, MSE: 86.59344222590867
svr, MSE: 88.57015141569659
forest, MSE: 21.560235683928042


In [117]:
print('for decorrelated data using ICA (validation scores):')
for name, model in models.items():
    model.fit(X_ica, y_train.squeeze())
    print('{}, MSE: {}'.format(name, mean_squared_error(model.predict(X_val_ica), y_val.squeeze())))

for decorrelated data using ICA (validation scores):
linear, MSE: 94.44968106049619
svr, MSE: 99.68177124966951
forest, MSE: 21.229186710086193


## Energy efficiency Data Set

In [135]:
df = pd.read_excel('../datasets/ENB2012_data.xlsx')

In [136]:
print('dataset size: {}'.format(df.shape))

dataset size: (768, 10)


In [137]:
subject = df['X6'].unique().tolist()
subject_map = dict(zip(subject, range(len(subject))))
subject_binary = np.zeros((df.shape[0], len(subject)))
for k, i in df.iterrows():
    subject_binary[k, subject_map[i['X6']]] = 1

In [138]:
# replacing categorical features with binary values
y = df.Y1.values
X = df.drop(['Y1', 'Y2', 'X6'], axis=1).values
X = np.concatenate([subject_binary, X], axis=1)

In [139]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

In [140]:
print('dataset size after preprocessing: {}'.format(X.shape))

dataset size after preprocessing: (768, 11)


### Training model with ICA-processed data

In [141]:
# number of components is fixed to 4
ica = FastICA(n_components=4)
X_ica = ica.fit_transform(X_train)
X_val_ica = ica.transform(X_val)

In [142]:
X_ica, y_train = shuffle(X_ica, y_train)

### * Estimating SVR penalty

In [143]:
eps_hyperopt = hyperopt_optimization_svr(X_ica, y_train, cv=4, max_iter_svr=200, max_iter_opt=15)
eps_bayesian = bayesian_optimization_svr(X_ica, y_train, cv=4, max_iter_svr=200, max_iter_opt=15)

D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00567543]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 50, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00190054]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 53, 'nit': 5, 'w

In [144]:
eps_opt = evaulate_best_svr_argument(X_ica, y_train, [eps_hyperopt, eps_bayesian], cv=6)

### * Estimating LightGBM params

In [145]:
params_opt = bayesian_optimization_lightgbm(X_ica, y_train, cv=6, max_iter_opt=50)

In [146]:
params_opt['bagging_freq'] = int(params_opt['bagging_freq'])
params_opt['n_estimators'] = int(params_opt['n_estimators'])

### * Evaluate with CV

In [147]:
models = {
    'linear': LinearRegression(),
    'svr': LinearSVR(C=100, epsilon=eps_opt, max_iter=200),
    'forest': LGBMRegressor(boosting_type='rf', **params_opt)
}

In [148]:
stats = {}

for k, model in models.items():
    stats[k] = []
    kfold = KFold(n_splits=8, shuffle=True)
    
    for train_ix, test_ix in kfold.split(X_ica, y_train):
        X_crossval_train, X_crossval_test = X_ica[train_ix], X_ica[test_ix]
        y_crossval_train, y_crossval_test = y_train[train_ix], y_train[test_ix]
        
        # here must be sume sort of optimization
        model.fit(X_crossval_train, y_crossval_train.ravel())
        stats[k].append(mean_squared_error(model.predict(X_crossval_test), y_crossval_test))

In [149]:
print('for decorrelated data using ICA')
for model, model_stats in stats.items():
    print('{}, MSE: {}'.format(model, np.mean(model_stats)))

for decorrelated data using ICA
linear, MSE: 21.808096411622596
svr, MSE: 22.560766456743597
forest, MSE: 7.445662485421545


In [150]:
print('for decorrelated data using ICA (validation scores):')
for name, model in models.items():
    model.fit(X_train, y_train.squeeze())
    print('{}, MSE: {}'.format(name, mean_squared_error(model.predict(X_val), y_val.squeeze())))

for decorrelated data using ICA (validation scores):
linear, MSE: 97.03911852546887
svr, MSE: 519.3941636621415
forest, MSE: 109.79270790594153
